In [27]:
from PandaStation import *
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG



import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector
    )
import pydrake.perception as mut
import open3d as o3d


def add_ycb(station, name, pose):
    ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
        "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
        "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
        "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
        "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
        "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
        "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}
    station.AddManipulandFromFile(ycb[name], pose)

In [14]:
class ProcessPointcloud(LeafSystem):
    
    def __init__(self, station, v):
        LeafSystem.__init__(self)
        
        camera_left_pcd = self.DeclareAbstractInputPort("camera_left_pcd", AbstractValue.Make(mut.PointCloud()))
        camera_middle_pcd = self.DeclareAbstractInputPort("camera_middle_pcd", AbstractValue.Make(mut.PointCloud()))
        camera_right_pcd = self.DeclareAbstractInputPort("camera_right_pcd", AbstractValue.Make(mut.PointCloud()))
        self.cameras = {'camera_left': camera_left_pcd,
                       'camera_middle': camera_middle_pcd,
                       'camera_right': camera_right_pcd}
        
        self.station = station
        self.plant = self.station.get_multibody_plant()
        self.plant_context = plant.GetMyContextFromRoot(station_context)
        
        self.v = v # meshcat visualizer
        
        self.DeclareAbstractOutputPort("merged_pcd", 
                                       lambda: AbstractValue.Make(o3d.geometry.PointCloud()), 
                                       self.DoCalcOutput)
        
        self.DeclarePeriodicEvent(0.5, 0.25, event = PublishEvent(
                                            trigger_type = TriggerType.kPeriodic,
                                            callback = self.MyPublish))
    
    def process_table_point_cloud(self, context):
        
        margin = 0.001
        # defines space above table
        crop_min = np.array([0.25, -0.5, 0.05+margin])
        crop_max = np.array([1.25, 0.5, 0.5])
        
        merged_pcd = o3d.geometry.PointCloud()
        
        for name,port in list(self.cameras.items()):
            pcd = self.EvalAbstractInput(context, port.get_index()).get_value()
            pcd = create_open3d_point_cloud(pcd)
            
            pcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound = crop_min, max_bound = crop_max))
            pcd.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius = 0.1, max_nn = 30))
            camera = self.plant.GetModelInstanceByName(name)
            body = self.plant.GetBodyByName("base", camera)
            X_C = self.plant.EvalBodyPoseInWorld(self.plant_context, body)
            pcd.orient_normals_towards_camera_location(X_C.translation())
            
            merged_pcd +=pcd
            
        return merged_pcd.voxel_down_sample(voxel_size = 0.005)
    
    def MyPublish(self, context, event):
        time = context.get_time()
        if not np.isclose(time, 0.25):
            return
        pcd = self.process_table_point_cloud(context)
        #print("time:", time)
        draw_open3d_point_cloud(self.v.vis["cloud"], pcd, size=0.003)
    
    def DoCalcOutput(self, context, output):
        pcd = self.process_table_point_cloud(context)
        output.set_value(pcd)
        
        
class FindGrasp(LeafSystem):
    
    def __init__(self, internal_station):
        LeafSystem.__init__(self)
        
        self.nq = 7
        
        self.merged_pcd_input_port = self.DeclareAbstractInputPort("merged_pcd", AbstractValue.Make(o3d.geometry.PointCloud()))   
        self.panda_grasp_position_output_port = self.DeclareVectorOutputPort("panda_grasp_position", BasicVector(self.nq),
                                                                self.DoCalcOutput)
        
        
        self.plant = internal_station.get_multibody_plant()
        self.panda = self.plant.GetModelInstanceByName("panda")
        self.station_context = internal_station.CreateDefaultContext()
        self.plant_context = self.plant.GetMyContextFromRoot(self.station_context)
        self.scene_graph = internal_station.get_scene_graph()
        self.scene_graph_context = internal_station.GetSubsystemContext(self.scene_graph, self.station_context)
        self.query_output_port = self.scene_graph.GetOutputPort("query")
        
        self.rng = np.random.default_rng()
        self.q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])
        self.max_tries = 1000
        
        self.p_tol = 0.01*np.ones(3)
        self.theta_tol = 0.01
        
    def grasp_candidate_cost(self, X_G, cloud):
        
        body = self.plant.GetBodyByName("panda_hand")

        X_GW = X_G.inverse()
        pts = np.asarray(cloud.points).T
        p_GC = X_GW.multiply(pts)

        # Crop to a region inside of the finger box.
        crop_min = [-0.009, -0.035, 0.06]
        crop_max = [0.009, 0.035, 0.115]
        indices = np.all((crop_min[0] <= p_GC[0,:], p_GC[0,:] <= crop_max[0],
                          crop_min[1] <= p_GC[1,:], p_GC[1,:] <= crop_max[1],
                          crop_min[2] <= p_GC[2,:], p_GC[2,:] <= crop_max[2]), 
                         axis=0)


        query_object = self.scene_graph.get_query_output_port().Eval(
                self.scene_graph_context)
        
        # Check collisions between the gripper and the point cloud
        margin = 0.0  # must be smaller than the margin used in the point cloud preprocessing.
        for pt in cloud.points:
            distances = query_object.ComputeSignedDistanceToPoint(pt, threshold=margin)
            if distances:
                #print('\ncollision with point cloud')
                return np.inf

        n_GC = X_GW.rotation().multiply(np.asarray(cloud.normals)[indices,:].T)

        # Penalize deviation of the gripper from vertical.
        # weight * -dot([0, 0, -1], R_G * [0, 1, 0]) = weight * R_G[2,1]
        cost = 20.0*X_G.rotation().matrix()[2, 1]

        # Reward sum |dot product of normals with gripper x|^2
        cost -= np.sum(n_GC[0,:]**2)

        return cost


    def generate_grasp_candidate_antipodal(self, cloud):
        """
        Picks a random point in the cloud, and aligns the robot finger with the normal of that pixel. 
        The rotation around the normal axis is drawn from a uniform distribution over [min_roll, max_roll].
        """
        body = self.plant.GetBodyByName("panda_hand")

        index = self.rng.integers(0,len(cloud.points)-1)

        # Use S for sample point/frame.
        p_WS = np.asarray(cloud.points[index])
        n_WS = np.asarray(cloud.normals[index])

        if not np.isclose(np.linalg.norm(n_WS), 1.0):
            return np.inf, None, None

        Gy = n_WS # gripper y axis aligns with normal
        # make orthonormal z axis, aligned with world down
        z = np.array([0.0, 0.0, -1.0])
        if np.abs(np.dot(z,Gy)) < 1e-6:
            # normal was pointing straight down.  reject this sample.
            #print('here')
            return np.inf, None, None

        Gz = z - np.dot(z,Gy)*Gy
        Gx = np.cross(Gy, Gz)
        R_WG = RotationMatrix(np.vstack((Gx, Gy, Gz)).T)
        p_GS_G = [0, 0.029, 0.1]

        # Try orientations from the center out
        min_pitch=-np.pi/3.0
        max_pitch=np.pi/3.0
        alpha = np.array([0.5, 0.65, 0.35, 0.8, 0.2, 1.0, 0.0])
        thetas = (min_pitch + (max_pitch - min_pitch)*alpha)
        warm_start = self.q_nominal
        for theta in thetas[:1]: 
            #print(f"trying angle {theta}")
            # Rotate the object in the hand by a random rotation (around the normal).
            R_WG2 = R_WG.multiply(RotationMatrix.MakeYRotation(theta))

            # Use G for gripper frame.
            p_SG_W = - R_WG2.multiply(p_GS_G)
            p_WG = p_WS + p_SG_W 

            X_G = RigidTransform(R_WG2, p_WG)
            
            #ik
            q = self.find_q(X_G, warm_start)

            if q is None:
                continue

            warm_start =  q
            
            self.plant.SetPositions(self.plant_context, self.panda, q)
            self.plant.SetPositions(self.plant_context, self.plant.GetModelInstanceByName("hand"), [-0.04, 0.04])

            cost = self.grasp_candidate_cost(X_G, cloud)
            if np.isfinite(cost):
                return cost, X_G, q

        return np.inf, None, None
    
    def find_q(self, X_G, warm_start =np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])):

        ik = InverseKinematics(self.plant, self.plant_context)
        ik.AddPositionConstraint(
                self.plant.GetFrameByName("panda_hand"),
                np.zeros(3),
                self.plant.world_frame(),
                X_G.translation() - self.p_tol,
                X_G.translation() + self.p_tol)
        ik.AddOrientationConstraint(
                self.plant.GetFrameByName("panda_hand"),
                RotationMatrix(),
                self.plant.world_frame(),
                X_G.rotation(),
                self.theta_tol)
        ik.AddMinimumDistanceConstraint(0.01, 0.1)
        q = ik.q()
        prog = ik.prog()
        q_nom = np.concatenate((self.q_nominal, np.zeros(2)))
        prog.AddQuadraticErrorCost(np.identity(len(q)), q_nom, q)
        q_warm = np.concatenate((warm_start, np.zeros(2)))
        prog.SetInitialGuess(q, q_warm)

        result = Solve(prog)

        if not result.is_success():
            return None

        return result.GetSolution(q)[:-2]    
    
    def DoCalcOutput(self, context, output):
        
        merged_pcd = self.EvalAbstractInput(context, self.merged_pcd_input_port.get_index()).get_value()
        cost = np.inf
        X_G = None
        q = None
        
        for i in range(self.max_tries):
            print(f"try number {i+1}", end = '')
            cost, X_G, q = self.generate_grasp_candidate_antipodal(merged_pcd)
            if np.isfinite(cost):
                print(f"FOUND Q: {q}")
                print(f"X_G: {X_G}")
                break
            print("", end = "\r")
            
        assert np.isfinite(cost)
        
        #print(q)
        output.set_value(q)
        

class Picker(LeafSystem):
    
    def __init__(self, internal_station):
        LeafSystem.__init__(self)
        
        self.nq = 7
        
        self.grasp_position_input_port = self.DeclareVectorInputPort("grasp_position", BasicVector(self.nq))
        self.panda_position_input_port = self.DeclareVectorInputPort("panda_position", 
                BasicVector(self.nq))
        self.panda_position_command_output_port = self.DeclareVectorOutputPort(
                "panda_position_command", BasicVector(self.nq),
                self.DoCalcOutput)
        
        self.plant = internal_station.get_multibody_plant()
        self.panda = self.plant.GetModelInstanceByName("panda")
        self.station_context = internal_station.CreateDefaultContext()
        self.plant_context = self.plant.GetMyContextFromRoot(self.station_context)
        self.scene_graph = internal_station.get_scene_graph()
        self.scene_graph_context = internal_station.GetSubsystemContext(self.scene_graph, self.station_context)
        self.query_output_port = self.scene_graph.GetOutputPort("query")
        
        self.initialized = False
        self.q_grasp = None
        
        
    def DoCalcOutput(self, context, output):
        time = context.get_time()
        
        if time < 0.5: # allow time for objects to settle
            q_start = self.EvalVectorInput(context, self.panda_position_input_port.get_index()).get_value()
            output.set_value(q_start)
            return
        
        if not self.initialized:
            q_start = self.EvalVectorInput(context, self.panda_position_input_port.get_index()).get_value()
            self.q_grasp = self.EvalVectorInput(context, self.grasp_position_input_port.get_index()).get_value()
            self.initialized = True
        
        assert self.q_grasp is not None
        
        output.set_value(self.q_grasp)
        
def make_internal_station():
    builder = DiagramBuilder()
    station = builder.AddSystem(PandaStation(0.001))
    station.SetupTableStation()
    station.Finalize()
    
    return station

In [29]:
builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation(time_step = 0.001))
station.SetupTableStation()


add_ycb(station, "brick", RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0.6, 0, 0.15]))
#station.AddManipulandFromFile("drake/examples/manipulation_station/models/061_foam_brick.sdf",
#    RigidTransform(RotationMatrix(), [0.75, 0, 0.1]))


plant = station.get_multibody_plant()
panda = plant.GetModelInstanceByName("panda")


station.Finalize()

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)

start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))


meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions


processor = builder.AddSystem(ProcessPointcloud(station, meshcat))
builder.Connect(station.GetOutputPort("camera_left_point_cloud"), processor.GetInputPort("camera_left_pcd"))
builder.Connect(station.GetOutputPort("camera_middle_point_cloud"), processor.GetInputPort("camera_middle_pcd"))
builder.Connect(station.GetOutputPort("camera_right_point_cloud"), processor.GetInputPort("camera_right_pcd"))

internal_station = make_internal_station()

grasp_finder = builder.AddSystem(FindGrasp(internal_station))
picker = builder.AddSystem(Picker(internal_station))

builder.Connect(processor.GetOutputPort("merged_pcd"), grasp_finder.GetInputPort("merged_pcd"))
builder.Connect(grasp_finder.GetOutputPort("panda_grasp_position"), picker.GetInputPort("grasp_position"))
builder.Connect(station.GetOutputPort("panda_position_measured"), picker.GetInputPort("panda_position"))
builder.Connect(picker.GetOutputPort("panda_position_command"), station.GetInputPort("panda_position"))


diagram = builder.Build()

#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation
#station.GetInputPort("panda_position").FixValue(station_context, [0.0, 0.1, 0.0, -1.2, 0, 1.6, 0])


meshcat.start_recording()

simulator.AdvanceTo(1)

meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6049...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7049/static/
Connected to meshcat-server.
try number 7FOUND Q: [ 0.05377934  0.59758421  0.009182   -1.90440905 -0.00676028  2.49203087
  0.06633382]
X_G: RigidTransform(
  R=RotationMatrix([
    [0.9999998963809359, 7.011268552436291e-07, -3.1870974867798966e-10],
    [7.011268552436291e-07, -0.9999998963809358, 0.0004545678335810397],
    [-5.169878828456423e-26, -0.00045456788068293816, -0.9999997933680419],
  ]),
  p=[0.6170905992265688, 0.029355537336874488, 0.1711065537189781],
)
